In [ ]:
# 특정 웹사이트에서 실시간 검색어 10가지를 가져오는 코드 작성 (Jupyter Notebook 사용)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import telegram
from telegram import Bot
import asyncio
import schedule

# Signal.bz 뉴스 페이지에서 실시간 검색어 10가지를 가져오기
URL = "https://signal.bz/news"

# 텔레그램 설정
API_TOKEN = '7936899167:AAFl_3jTqQ-B2b4CUPaKnf_4tWgoY1FI_os'  # 실제 봇 API 토큰으로 변경하세요
CHAT_ID = '1059657134'  # 메시지를 보낼 채팅 ID를 입력하세요

# 실시간 검색어 가져오기 함수 (Selenium 사용)
def get_realtime_search_terms(url):
    print(f"[{datetime.datetime.now()}] 실시간 검색어 수집을 시작합니다.")
    try:
        # 크롬 드라이버 설정
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # 브라우저 창을 표시하지 않고 실행
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        driver.get(url)
        print("웹페이지에 접속했습니다.")
        
        # 실시간 검색어 요소 가져오기
        search_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/main/div/section/div/section/section[1]/div[2]/div/div/div/a/span[2]")
        terms = [element.text.strip() for element in search_elements[:10]]
        
        driver.quit()
        if terms:
            print(f"[{datetime.datetime.now()}] 실시간 검색어 수집 성공: {terms}")
        else:
            print(f"[{datetime.datetime.now()}] 실시간 검색어를 수집하지 못했습니다.")
        return terms
    except Exception as e:
        print(f"[{datetime.datetime.now()}] 실시간 검색어 수집 중 오류 발생: {e}")
        return []

# 인기 뉴스 가져오기 함수 (Selenium 사용)
def get_popular_news(url):
    print(f"[{datetime.datetime.now()}] 인기 뉴스 수집을 시작합니다.")
    try:
        # 크롬 드라이버 설정
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # 브라우저 창을 표시하지 않고 실행
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        driver.get(url)
        print("웹페이지에 접속했습니다.")
        
        # 인기 뉴스 요소 가져오기
        popular_news_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/main/div/section/div/section/section[3]/div/section/div/div/div/div[2]")
        popular_news = [element.text.strip() for element in popular_news_elements[:10]]
        
        driver.quit()
        if popular_news:
            print(f"[{datetime.datetime.now()}] 인기 뉴스 수집 성공: {popular_news}")
        else:
            print(f"[{datetime.datetime.now()}] 인기 뉴스를 수집하지 못했습니다.")
        return popular_news
    except Exception as e:
        print(f"[{datetime.datetime.now()}] 인기 뉴스 수집 중 오류 발생: {e}")
        return []

# 검색어 출력 및 저장 함수
def save_search_terms(terms, popular_news):
    print(f"[{datetime.datetime.now()}] 검색어 및 인기 뉴스 저장을 시작합니다.")
    if terms:
        # 실시간 검색어 데이터프레임으로 변환
        df = pd.DataFrame(terms, columns=['실시간 검색어'])
        # 데이터 저장 (파일명에 현재 날짜 포함)
        current_date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"실검top10_{current_date}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"검색어가 파일로 저장되었습니다: {filename}")
        # 데이터프레임 출력
        display(df)
    else:
        print("검색어를 가져올 수 없습니다.")
    
    if popular_news:
        # 인기 뉴스 데이터프레임으로 변환
        df_popular = pd.DataFrame(popular_news, columns=['인기 뉴스'])
        # 데이터 저장 (파일명에 현재 날짜 포함)
        filename_popular = f"인기뉴스_{current_date}.csv"
        df_popular.to_csv(filename_popular, index=False, encoding='utf-8-sig')
        print(f"인기 뉴스가 파일로 저장되었습니다: {filename_popular}")
        # 데이터프레임 출력
        display(df_popular)
    else:
        print("인기 뉴스를 가져올 수 없습니다.")

# 텔레그램으로 메시지 전송 함수
def send_message(api_token, chat_id, message):
    print(f"[{datetime.datetime.now()}] 텔레그램 메시지 전송을 시작합니다.")
    bot = Bot(token=api_token)
    try:
        asyncio.run(bot.send_message(chat_id=chat_id, text=message)) if not asyncio.get_event_loop().is_running() else asyncio.ensure_future(bot.send_message(chat_id=chat_id, text=message))
        print(f"[{datetime.datetime.now()}] 메시지가 성공적으로 전송되었습니다.")
    except telegram.error.TelegramError as e:
        print(f"[{datetime.datetime.now()}] 메시지 전송 실패: {e}")

# 주기적으로 실행하는 함수
def job():
    print(f"[{datetime.datetime.now()}] 주기적 작업 실행 시작")
    current_hour = datetime.datetime.now().hour
    # 한국 시간 기준으로 오후 11시부터 오전 9시까지는 전송하지 않음
    if current_hour >= 0 or current_hour < 9:
        print(f"[{datetime.datetime.now()}] 현재는 메시지를 보내지 않는 시간입니다.")
        return
    
    realtime_terms = get_realtime_search_terms(URL)
    popular_news = get_popular_news(URL)
    save_search_terms(realtime_terms, popular_news)
    
    # 텔레그램으로 실시간 검색어와 인기 뉴스 전송
    if realtime_terms:
        message = "실시간 검색어:\n" + "\n".join(realtime_terms)
        send_message(API_TOKEN, CHAT_ID, message)
    
    if popular_news:
        message = "인기 뉴스:\n" + "\n".join(popular_news)
        send_message(API_TOKEN, CHAT_ID, message)

# 스케줄 설정
schedule.every(59).minutes.do(job)

if __name__ == "__main__":
    while True:
        print(f"[{datetime.datetime.now()}] 스케줄 확인 중...")
        schedule.run_pending()
        time.sleep(60)  # 1분마다 스케줄 확인


[2024-11-09 21:05:31.218915] 스케줄 확인 중...
[2024-11-09 21:06:31.220015] 스케줄 확인 중...
[2024-11-09 21:07:31.220533] 스케줄 확인 중...
[2024-11-09 21:08:31.221042] 스케줄 확인 중...
[2024-11-09 21:09:31.221387] 스케줄 확인 중...
[2024-11-09 21:10:31.221678] 스케줄 확인 중...
[2024-11-09 21:11:31.225336] 스케줄 확인 중...
[2024-11-09 21:12:31.227120] 스케줄 확인 중...
[2024-11-09 21:13:31.227480] 스케줄 확인 중...
[2024-11-09 21:14:31.229919] 스케줄 확인 중...
[2024-11-09 21:15:31.230506] 스케줄 확인 중...
[2024-11-09 21:16:31.230826] 스케줄 확인 중...
[2024-11-09 21:17:31.232667] 스케줄 확인 중...
[2024-11-09 21:18:31.233702] 스케줄 확인 중...
[2024-11-09 21:19:31.234234] 스케줄 확인 중...
[2024-11-09 21:20:31.234655] 스케줄 확인 중...
[2024-11-09 21:21:31.235017] 스케줄 확인 중...
[2024-11-09 21:22:31.235428] 스케줄 확인 중...
[2024-11-09 21:23:31.236009] 스케줄 확인 중...
[2024-11-09 21:24:31.236516] 스케줄 확인 중...
[2024-11-09 21:25:31.237045] 스케줄 확인 중...
[2024-11-09 21:26:31.237435] 스케줄 확인 중...
[2024-11-09 21:27:31.237784] 스케줄 확인 중...
[2024-11-09 21:28:31.238197] 스케줄 확인 중...
[2024-11-09 21:2

,실시간 검색어
0,다리미 패밀리
1,손흥민 토트넘
2,정년이
3,광안리 불꽃축제
4,김준수 김재중
5,국정농단 특검 촉구
6,신현빈
7,가보자고 김광수 대표
8,수원 전남
9,제1145회 로또 1등


인기 뉴스가 파일로 저장되었습니다: 인기뉴스_20241109_220458.csv


,인기 뉴스
0,"[단독]""윤석열 퇴진"" 민주노총 집회 과정서 10명 현행범 체포"
1,가을밤 화려하게 수놓은 부산불꽃축제…103만명 인파 '탄성'(종합)
2,"TSMC, 트럼프 의식했나…중국에 AI칩 공급 중단"
3,"""한국 여성처럼 결혼·출산 거부하자""…미국에 번진 韓페미니즘 '4B 운동'"
4,"美법무부 ""이란, 대선 전 또 트럼프 암살모의""…이란 ""사실무근""(종합)"
5,여수 최고급호텔 2층 사우나 화재…5명 연기흡입·99명 대피(종합)
6,"민주 ""과태료 미납 7회 박장범 후안무치…KBS 사장 선임 저지하겠다"""
7,세계식량가격 18개월 만에 최고…전달보다 2% 상승
8,현대트랜시스 파업 한달만에 종료…오는 11일부터 정상 출근(종합)
9,'그래미 어워즈'도 대세는 '女風'…K팝은 노미네이트 불발(종합)


[2024-11-09 22:04:58.382700] 텔레그램 메시지 전송을 시작합니다.
[2024-11-09 22:04:59.751179] 메시지가 성공적으로 전송되었습니다.
[2024-11-09 22:04:59.751179] 텔레그램 메시지 전송을 시작합니다.
[2024-11-09 22:05:01.194500] 메시지가 성공적으로 전송되었습니다.
[2024-11-09 22:06:01.196493] 스케줄 확인 중...
[2024-11-09 22:07:01.197393] 스케줄 확인 중...
[2024-11-09 22:08:01.198122] 스케줄 확인 중...
[2024-11-09 22:09:01.198529] 스케줄 확인 중...
[2024-11-09 22:10:01.199426] 스케줄 확인 중...
[2024-11-09 22:11:01.199764] 스케줄 확인 중...
[2024-11-09 22:12:01.200071] 스케줄 확인 중...
[2024-11-09 22:13:01.200427] 스케줄 확인 중...
[2024-11-09 22:14:01.201037] 스케줄 확인 중...
[2024-11-09 22:15:01.201735] 스케줄 확인 중...
[2024-11-09 22:16:01.201966] 스케줄 확인 중...
[2024-11-09 22:17:01.202547] 스케줄 확인 중...
[2024-11-09 22:18:01.202993] 스케줄 확인 중...
[2024-11-09 22:19:01.203386] 스케줄 확인 중...
[2024-11-09 22:20:01.203870] 스케줄 확인 중...
[2024-11-09 22:21:01.204573] 스케줄 확인 중...
[2024-11-09 22:22:01.205088] 스케줄 확인 중...
[2024-11-09 22:23:01.205549] 스케줄 확인 중...
[2024-11-09 22:24:01.206093] 스케줄 확인 중...
[2024-11-09 22:25:01.2064

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import telegram
from telegram import Bot
import asyncio
import schedule

current_time = datetime.datetime.now()
print(f"현재 시스템 시간: {current_time}")


# Signal.bz 뉴스 페이지에서 실시간 검색어 10가지를 가져오기
URL = "https://signal.bz/news"

# 텔레그램 설정
API_TOKEN = '7936899167:AAFl_3jTqQ-B2b4CUPaKnf_4tWgoY1FI_os'
CHAT_ID = '1059657134'

# 실시간 검색어 가져오기 함수 (Selenium 사용)
def get_realtime_search_terms(url):
    print(f"[{datetime.datetime.now()}] 실시간 검색어 수집을 시작합니다.")
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        driver.get(url)
        print("웹페이지에 접속했습니다.")
        
        search_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/main/div/section/div/section/section[1]/div[2]/div/div/div/a/span[2]")
        terms = [element.text.strip() for element in search_elements[:10]]
        
        driver.quit()
        if terms:
            print(f"[{datetime.datetime.now()}] 실시간 검색어 수집 성공: {terms}")
        else:
            print(f"[{datetime.datetime.now()}] 실시간 검색어를 수집하지 못했습니다.")
        return terms
    except Exception as e:
        print(f"[{datetime.datetime.now()}] 실시간 검색어 수집 중 오류 발생: {e}")
        return []

# 인기 뉴스 가져오기 함수 (Selenium 사용)
def get_popular_news(url):
    print(f"[{datetime.datetime.now()}] 인기 뉴스 수집을 시작합니다.")
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        driver.get(url)
        print("웹페이지에 접속했습니다.")
        
        popular_news_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/main/div/section/div/section/section[3]/div/section/div/div/div/div[2]")
        popular_news = [element.text.strip() for element in popular_news_elements[:10]]
        
        driver.quit()
        if popular_news:
            print(f"[{datetime.datetime.now()}] 인기 뉴스 수집 성공: {popular_news}")
        else:
            print(f"[{datetime.datetime.now()}] 인기 뉴스를 수집하지 못했습니다.")
        return popular_news
    except Exception as e:
        print(f"[{datetime.datetime.now()}] 인기 뉴스 수집 중 오류 발생: {e}")
        return []

# 검색어 출력 및 저장 함수
def save_search_terms(terms, popular_news):
    print(f"[{datetime.datetime.now()}] 검색어 및 인기 뉴스 저장을 시작합니다.")
    if terms:
        df = pd.DataFrame(terms, columns=['실시간 검색어'])
        current_date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"실검top10_{current_date}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"검색어가 파일로 저장되었습니다: {filename}")
        display(df)
    else:
        print("검색어를 가져올 수 없습니다.")
    
    if popular_news:
        df_popular = pd.DataFrame(popular_news, columns=['인기 뉴스'])
        filename_popular = f"인기뉴스_{current_date}.csv"
        df_popular.to_csv(filename_popular, index=False, encoding='utf-8-sig')
        print(f"인기 뉴스가 파일로 저장되었습니다: {filename_popular}")
        display(df_popular)
    else:
        print("인기 뉴스를 가져올 수 없습니다.")

# 텔레그램으로 메시지 전송 함수
async def send_message(api_token, chat_id, message):
    print(f"[{datetime.datetime.now()}] 텔레그램 메시지 전송을 시작합니다.")
    bot = Bot(token=api_token)
    try:
        await bot.send_message(chat_id=chat_id, text=message)
        print(f"[{datetime.datetime.now()}] 메시지가 성공적으로 전송되었습니다.")
    except telegram.error.TelegramError as e:
        print(f"[{datetime.datetime.now()}] 메시지 전송 실패: {e}")

# 주기적으로 실행하는 함수
def job():
    print(f"[{datetime.datetime.now()}] 주기적 작업 실행 시작")
    current_hour = datetime.datetime.now().hour
    if current_hour >= 1 or current_hour < 9:
        print(f"[{datetime.datetime.now()}] 현재는 메시지를 보내지 않는 시간입니다.")
        return
    
    realtime_terms = get_realtime_search_terms(URL)
    popular_news = get_popular_news(URL)
    save_search_terms(realtime_terms, popular_news)
    
    loop = asyncio.get_event_loop()
    
    if realtime_terms:
        message = "실시간 검색어:\n" + "\n".join(realtime_terms)
        loop.run_until_complete(send_message(API_TOKEN, CHAT_ID, message))
    
    if popular_news:
        message = "인기 뉴스:\n" + "\n".join(popular_news)
        loop.run_until_complete(send_message(API_TOKEN, CHAT_ID, message))

# 스케줄 설정 - 테스트를 위해 5분마다 실행
schedule.every(5).minutes.do(job)

if __name__ == "__main__":
    while True:
        print(f"[{datetime.datetime.now()}] 스케줄 확인 중...")
        schedule.run_pending()
        time.sleep(60)  # 1분마다 스케줄 확인


현재 시스템 시간: 2024-11-09 23:43:47.897864
[2024-11-09 23:43:47.898861] 스케줄 확인 중...
[2024-11-09 23:44:47.899821] 스케줄 확인 중...
[2024-11-09 23:45:47.900260] 스케줄 확인 중...
[2024-11-09 23:46:47.900532] 스케줄 확인 중...
[2024-11-09 23:47:47.900739] 스케줄 확인 중...
[2024-11-09 23:48:47.901173] 스케줄 확인 중...
[2024-11-09 23:48:47.901173] 주기적 작업 실행 시작
[2024-11-09 23:48:47.901173] 현재는 메시지를 보내지 않는 시간입니다.
[2024-11-09 23:49:47.901665] 스케줄 확인 중...
[2024-11-09 23:50:47.902264] 스케줄 확인 중...
[2024-11-09 23:51:47.902878] 스케줄 확인 중...
[2024-11-09 23:52:47.903523] 스케줄 확인 중...
[2024-11-09 23:53:47.905384] 스케줄 확인 중...
[2024-11-09 23:53:47.905384] 주기적 작업 실행 시작
[2024-11-09 23:53:47.905384] 현재는 메시지를 보내지 않는 시간입니다.
[2024-11-09 23:54:47.905777] 스케줄 확인 중...
[2024-11-09 23:55:47.906277] 스케줄 확인 중...
[2024-11-09 23:56:47.906948] 스케줄 확인 중...
[2024-11-09 23:57:47.907361] 스케줄 확인 중...
[2024-11-09 23:58:47.907887] 스케줄 확인 중...
[2024-11-09 23:58:47.907887] 주기적 작업 실행 시작
[2024-11-09 23:58:47.907887] 현재는 메시지를 보내지 않는 시간입니다.
[2024-11-09 23:59:47.908